# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
# import libraries
from sqlalchemy import create_engine
from sqlite3 import connect 
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
import re
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sashyabaral/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# load data from database
conn = connect('Project2DataBase.db') 
dfnbr = pd.read_sql('SELECT * FROM MyTable', conn) 
X = dfnbr['message'] 
y = dfnbr.drop(['id', 'message','original','genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:

def tokenize(text):
    """
    Tokenizes text that is passed in.
    
    Args:
        text (str): The text you want to tokenize.
        
    Returns:
        list: A list of tokens after processing.
    """
    text = text.lower()
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    tokens = word_tokenize(text)
    
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    
    tokens = [stemmer.stem(w) for w in tokens]
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    
    return tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.tree import DecisionTreeClassifier
pipeline =  Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
])
pipeline

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7ff881b33dc0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=DecisionTreeClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

/Users/sashyabaral/miniconda/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7ff881b33dc0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=DecisionTreeClassifier()))])

In [6]:
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
from sklearn.metrics import classification_report


y_pred = pipeline.predict(X_test)

for i, column in enumerate(y_test.columns):
    print(f"Classification Report for {column}:")
    print(classification_report(y_test[column], y_pred[:, i]))


Classification Report for related:
              precision    recall  f1-score   support

           0       0.48      0.45      0.47      1266
           1       0.82      0.84      0.83      3938
           2       0.46      0.60      0.52        40

    accuracy                           0.74      5244
   macro avg       0.59      0.63      0.61      5244
weighted avg       0.74      0.74      0.74      5244

Classification Report for request:
              precision    recall  f1-score   support

           0       0.91      0.93      0.92      4349
           1       0.61      0.56      0.58       895

    accuracy                           0.86      5244
   macro avg       0.76      0.74      0.75      5244
weighted avg       0.86      0.86      0.86      5244

Classification Report for offer:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy             

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5198
           1       0.06      0.07      0.06        46

    accuracy                           0.98      5244
   macro avg       0.52      0.53      0.53      5244
weighted avg       0.98      0.98      0.98      5244

Classification Report for shops:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5222
           1       0.04      0.05      0.04        22

    accuracy                           0.99      5244
   macro avg       0.52      0.52      0.52      5244
weighted avg       0.99      0.99      0.99      5244

Classification Report for aid_centers:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5177
           1       0.14      0.09      0.11        67

    accuracy                           0.98      5244
   macro avg       0.56      0.54      0.55      5244
we

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'clf__estimator__max_depth': [None, 10, 20],
    'clf__estimator__min_samples_split': [2, 5, 10],
    'clf__estimator__min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(pipeline, param_grid=param_grid, cv=3, n_jobs=-1)

grid_search.fit(X_train, y_train)

best_estimator = grid_search.best_estimator_
best_params = grid_search.best_params_




/Users/sashyabaral/miniconda/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/sashyabaral/miniconda/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/sashyabaral/miniconda/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/sashyabaral/miniconda/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/sashyabaral/miniconda/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is no

/Users/sashyabaral/miniconda/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/sashyabaral/miniconda/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/sashyabaral/miniconda/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/sashyabaral/miniconda/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/sashyabaral/miniconda/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is no

/Users/sashyabaral/miniconda/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/sashyabaral/miniconda/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
print("Best Parameters:")
print(best_params)

y_pred_grid = best_estimator.predict(X_test)

for i, column in enumerate(y_test.columns):
    print(f"Classification Report for {column}:")
    print(classification_report(y_test[column], y_pred_grid[:, i]))

Best Parameters:
{'clf__estimator__max_depth': 10, 'clf__estimator__min_samples_leaf': 1, 'clf__estimator__min_samples_split': 2}
Classification Report for related:
              precision    recall  f1-score   support

           0       0.58      0.34      0.43      1266
           1       0.81      0.92      0.86      3938
           2       0.50      0.03      0.05        40

    accuracy                           0.77      5244
   macro avg       0.63      0.43      0.44      5244
weighted avg       0.75      0.77      0.75      5244

Classification Report for request:
              precision    recall  f1-score   support

           0       0.90      0.96      0.93      4349
           1       0.74      0.50      0.60       895

    accuracy                           0.88      5244
   macro avg       0.82      0.73      0.76      5244
weighted avg       0.88      0.88      0.88      5244

Classification Report for offer:
              precision    recall  f1-score   support

    

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5198
           1       0.10      0.04      0.06        46

    accuracy                           0.99      5244
   macro avg       0.54      0.52      0.53      5244
weighted avg       0.98      0.99      0.99      5244

Classification Report for shops:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5222
           1       0.00      0.00      0.00        22

    accuracy                           1.00      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.99      1.00      0.99      5244

Classification Report for aid_centers:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5177
           1       0.23      0.04      0.07        67

    accuracy                           0.99      5244
   macro avg       0.61      0.52      0.53      5244
we

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [10]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
pipeline_2 =  Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])
pipeline_2

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7ff881b33dc0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier()))])

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline_2.fit(X_train, y_train)

/Users/sashyabaral/miniconda/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7ff881b33dc0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier()))])

In [12]:
from sklearn.metrics import classification_report


y_pred = pipeline_2.predict(X_test)

for i, column in enumerate(y_test.columns):
    print(f"Classification Report for {column}:")
    print(classification_report(y_test[column], y_pred[:, i]))


Classification Report for related:
              precision    recall  f1-score   support

           0       0.62      0.31      0.42      1266
           1       0.81      0.93      0.86      3938
           2       0.30      0.30      0.30        40

    accuracy                           0.78      5244
   macro avg       0.57      0.52      0.53      5244
weighted avg       0.76      0.78      0.75      5244

Classification Report for request:
              precision    recall  f1-score   support

           0       0.84      1.00      0.91      4349
           1       0.82      0.09      0.17       895

    accuracy                           0.84      5244
   macro avg       0.83      0.54      0.54      5244
weighted avg       0.84      0.84      0.79      5244

Classification Report for offer:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy             

/Users/sashyabaral/miniconda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sashyabaral/miniconda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sashyabaral/miniconda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sashya

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4916
           1       0.56      0.02      0.03       328

    accuracy                           0.94      5244
   macro avg       0.75      0.51      0.50      5244
weighted avg       0.91      0.94      0.91      5244

Classification Report for transport:
              precision    recall  f1-score   support

           0       0.95      1.00      0.98      5004
           1       0.50      0.00      0.01       240

    accuracy                           0.95      5244
   macro avg       0.73      0.50      0.49      5244
weighted avg       0.93      0.95      0.93      5244

Classification Report for buildings:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      4977
           1       0.77      0.04      0.07       267

    accuracy                           0.95      5244
   macro avg       0.86      0.52      0.52      5244


/Users/sashyabaral/miniconda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sashyabaral/miniconda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sashyabaral/miniconda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sashya

### 9. Export your model as a pickle file

In [13]:
import pickle
pickle.dump(grid_search, open("gridsearchcv_model", 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.